# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [1]:
import bac.simulate.gromacs as gmx

In [2]:
em_steep = gmx.Run(integrator='steep', number_of_steps=5000)

In [3]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10

In [4]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.non_bonded_controller.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 1
nbc.neighbor_list_type = gmx.non_bonded_controller.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.non_bonded_controller.PeriodicBoundaryConditionType.xyz
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.non_bonded_controller.CoulombType.pme
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_type = gmx.non_bonded_controller.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.non_bonded_controller.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.non_bonded_controller.LongRangeDispersionCorrectionType.energy_and_pressure

In [15]:
fe = gmx.FreeEnergyController()

fe.initial_lambda_state = 0
fe.delta_lambda = 0
fe.calculate_lambda_neighbors = 1
fe.van_der_waals_lambdas = [round(x*0.05, 2) for x in range(21)] 
fe.coulomb_lambdas = [0.0]*20
fe.bonded_lambdas = [0.0]*20
fe.restraint_lambdas =[0.0]*20
fe.mass_lambdas = [0.0]*20
fe.temperature_lambdas = [0.0]*20

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

em_steep.free_energy_controller = fe

In [6]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

In [7]:
from bac.simulate.coding import Encoder, Engine

In [16]:
Encoder.encode(em_steep, engine=Engine.gromacs, path='/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/em_steep.mdp')

In [10]:
a = 3*0.05

In [14]:
round(a, 3)

0.15